In [2]:
import datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
from scipy.stats import hmean
import numpy as np
from scipy.stats import sem, hmean, ks_2samp

import sys
sys.path.append('..')
from aggregate_eval_stat import get_forget_quality, get_model_utility

/home/zhilif/anaconda3/envs/tf/lib/python3.9/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
import glob
import os 
import copy 
import pandas as pd
import json 
from natsort import natsorted 
wds = ['0.01']
model_family=['llama2-7b', 'phi']
lr_map = {'llama2-7b': '1e-05', 'phi': '2e-05'}
ft_base_dir = '../paper_models/final_ft_noLORA_5_epochs_inst_lr{}_{}_full_wd{}/checkpoint-625'
retain_base_dir = '../paper_models/final_ft_noLORA_5_epochs_inst_lr{}_{}_{}_wd{}'
unlearn_algs = ['grad_ascent', 'grad_diff', 'idk', 'KL']
forget_ratios=['01', '05', '10']
df_cols = ["ROUGE Real Authors", "Prob. Real Authors", "Truth Ratio Real Authors", \
    "ROUGE Real World", "Prob. Real World", "Truth Ratio Real World", "ROUGE Retain", \
    "Prob. Retain", "Truth Ratio Retain", "ROUGE Forget", "Prob. Forget", \
    "Truth Ratio Forget", "Model Utility", "Forget Quality", "Method", "Submitted By",
    "Epoch"
]
for model in model_family:
    lr = lr_map[model]
    for forget_ratio in forget_ratios:
        df = pd.DataFrame(columns=['Method'])

        for wd in wds:
            for unlearn_alg in unlearn_algs:
                ft_dir = ft_base_dir.format(lr, model, wd)
                unlearn_result_dir = f'{ft_dir}/{unlearn_alg}_1e-05_forget{forget_ratio}'
                # get all subfolder with name checkpoint in unlearn_result_dir
                unlearn_res_files  = glob.glob(f'{unlearn_result_dir}/checkpoint-*/eval_results/ds_size300/eval_log_aggregated.json')
                unlearn_res_files = natsorted(unlearn_res_files)
                retain_ratio = int(100-int(forget_ratio))
                print(retain_ratio)
                retain_dir = retain_base_dir.format(lr, model, f'retain{retain_ratio}', wd)
                retain_res_file = glob.glob(f'{retain_dir}/checkpoint-*/eval_results/ds_size300/eval_log_aggregated.json')
                # Filter out only directories
                # subfolders = [f for f in ckpt_dirs if os.path.isdir(f)]
                # print(unlearn_result_dir)
                # print(retain_res_file)
                retain_res = json.load(open(retain_res_file[0]))
                for i, ckpt_file in enumerate(unlearn_res_files):
                    ckpt_res = json.load(open(ckpt_file))
                    model_utility = get_model_utility(ckpt_res)
                    forget_quality = get_forget_quality(ckpt_res, retain_res)
                    # print(ckpt_file, forget_quality)
                    df_row = copy.deepcopy(model_utility)
                    df_row['Forget Quality'] = forget_quality['Forget Quality']
                    df_row['Method'] = unlearn_alg
                    df_row['Submitted By'] = 'Baseline'
                    df_row['Epoch'] = int(i)
                    df = pd.concat([df, pd.DataFrame([df_row])], ignore_index=True)
        
        df.to_csv(f'{model}_{forget_ratio}.csv', index=False)

                

99
99
99
99
95
95
95
95
90
90
90
90
99
99
99
99
95
95
95
95
90
90
90
90


In [20]:
import numpy as np
# load json
with open('/home/zhilif/tofu/debug/phi_retain90/checkpoint-0/eval_log_aggregated.json') as f:
    data2 = json.load(f)

get_model_utility(data)
get_forget_quality(data, data2)

{'Forget Quality': 0.5568991019617135,
 'KS Test PVal Forget': 0.5568991019617135,
 'KS Test Forget': 0.11}

In [ ]:
# load json to panda 
import pandas as pd
df = pd.read_json('/home/zhilif/tofu/paper_models/final_ft_noLORA_5_epochs_inst_lr2e-05_phi_full/checkpoint-625/grad_ascent_1e-05_forget01/checkpoint-5/eval_results/ds_size300/eval_log_aggregated.json')

In [2]:
# iterate through the df['eval_log.json']
for k, v in df['eval_log.json'].items():
    print(k, len(v.keys()))

for k, v in df['eval_real_author_wo_options.json'].items():
    print(k, len(v.keys()))

for k, v in df['eval_real_world_wo_options.json'].items():
    print(k, len(v.keys()))

for k, v in df['eval_log_forget.json'].items():
    print(k, len(v.keys()))

avg_gt_loss 300
gt_loss 300
num_token_gt 300
generated_text 300
rouge1_recall 300
rougeL_recall 300
average_perturb_loss 300
avg_paraphrased_loss 300
truth_ratio 300
paraphrased_loss 300
perturb_loss 300
num_token_paraphrased 300
num_token_perturb 300
avg_gt_loss 100
gt_loss 100
num_token_gt 100
generated_text 100
rouge1_recall 100
rougeL_recall 100
average_perturb_loss 100
avg_paraphrased_loss 100
truth_ratio 100
paraphrased_loss 100
perturb_loss 100
num_token_paraphrased 100
num_token_perturb 100
avg_gt_loss 117
gt_loss 117
num_token_gt 117
generated_text 117
rouge1_recall 117
rougeL_recall 117
average_perturb_loss 117
avg_paraphrased_loss 117
truth_ratio 117
paraphrased_loss 117
perturb_loss 117
num_token_paraphrased 117
num_token_perturb 117
avg_gt_loss 40
gt_loss 40
num_token_gt 40
generated_text 40
rouge1_recall 40
rougeL_recall 40
average_perturb_loss 40
avg_paraphrased_loss 40
truth_ratio 40
paraphrased_loss 40
perturb_loss 40
num_token_paraphrased 40
num_token_perturb 40


In [8]:
import os 
from pathlib import Path 

model_family = ['llama2-7b', 'phi']
splits = ['full', 'retain90', 'retain95', 'retain99']
wds = [0, 0.01]
lr_map = {
    'llama2-7b': '1e-05',
    'phi': '2e-05'
}
ckpt_dict = {
    'full': 625,
    'retain90': 562,
    'retain95': 593,
    'retain99': 618
}
for model in model_family:
    lr = lr_map[model]
    for split in splits:
        for wd in wds:
            model_name = f'ft_epoch5_lr{lr}_{model}_{split}_wd{wd}'
            model_path = f'/home/zhilif/tofu/paper_models/{model_name}/checkpoint-{ckpt_dict[split]}'
            eval_results = f'{model_path}/eval_results'
            target_path = f'/home/zhilif/tofu/data/{model_name}/'
            Path(target_path).mkdir(parents=True, exist_ok=True)
            # copy result_path to /home/zhilif/tofu/data
            os.system(f'cp -r {eval_results} {target_path}')
            